# 5. Value Betting Calculator

In this notebook we'll make use of our match dataset and the Logistc Regression model we've built in the last stage, to predict the chances for each player to win in any given match.

We'll then compare these chances with the odds available for those matches, and build a Dataframe where each row will contain information regarding which player we would have to bet on!



trying to simulate bets on matches where we think to have an edge on the betting market (i.e. a value bet occurs when the real odds for an event are higher than the odds "predicted" by our model).

Finally, we'll compare several betting strategies and we'll find out if it's possible to make a good ROI backtesting our betting strategies.

Let's import the libraries needed for this notebook:

In [1]:
import pandas as pd
from math import exp
import pickle

Now, we'll load the Logistc Regression model trained in last notebook:

In [2]:
lreg = pickle.load(open("logistic_regression.lr", 'rb'))

Let's take a look at the coefficients and the intercept of this model:

In [3]:
lreg.coef_

array([[-0.00820549,  0.01828819,  0.0182351 , -0.00366707, -0.03013789,
         0.02565256,  0.01003487,  0.01715352, -0.01469755,  0.0393068 ,
        -0.04171493, -0.00705053,  0.01603137, -0.07591471,  0.01989064,
        -0.03044798,  0.08453897, -0.01250299,  0.01553793,  0.36270172,
        -0.2729425 ,  0.27082261,  0.31381609, -0.5232965 , -2.46044923,
         2.23443672, -3.02157641,  3.38983883]])

In [4]:
lreg.intercept_

array([0.01187192])

Now, let's open the Coefficients dataframe saved in the last notebook, and let's create a dictionary object based on the dataframe, where the key is going to be the attribute name and the value will be the attribute's coefficient:

In [5]:
coefs_df = pd.read_csv("csv/Coefficients.csv")

In [6]:
coefs_df.head()

,Column,Coef
0,Indoor,-0.008205
1,Outdoor,0.018288
2,Carpet,0.018235
3,Clay,-0.003667
4,Grass,-0.030138


In [7]:
coefs_map = {}
for ix, row in coefs_df.iterrows():

    attribute = row['Column']
    attribute_coef = row['Coef']

    coefs_map[attribute] = attribute_coef

In [8]:
coefs_map

{'Indoor': -0.008205490055114922,
 'Outdoor': 0.018288190776828448,
 'Carpet': 0.018235098719900585,
 'Clay': -0.0036670657947837455,
 'Grass': -0.030137893520682982,
 'Hard': 0.02565256131728023,
 'ATP250': 0.010034866580374424,
 'ATP500': 0.017153517648049232,
 'Grand Slam': -0.014697553313709538,
 'Masters 1000': 0.03930679973658772,
 'Masters Cup': -0.04171492992958769,
 '1st Round': -0.007050532204758627,
 '2nd Round': 0.0160313689626121,
 '3rd Round': -0.07591470921881736,
 '4th Round': 0.019890644865569376,
 'Quarterfinals': -0.03044797716182537,
 'Round Robin': 0.08453897012931598,
 'Semifinals': -0.012502992714257024,
 'The Final': 0.01553792806387606,
 'Rank Index': 0.3627017152187937,
 'Pl0 Recent Form': -0.27294250215186305,
 'Pl0 Form': 0.2708226143231703,
 'Pl1 Recent Form': 0.3138160892166305,
 'Pl1 Form': -0.5232964971778234,
 'Pl0 Perf. vs Similar Opponent': -2.4604492270974863,
 'Pl1 Perf. vs Similar Opponent': 2.234436723553626,
 'Pl0 Surface Performance': -3.0215764

Let's make a quick analysis on the greatest indicators for player 1 to win a match against player 0.
Let's see which attributes matter the most:

In [9]:
coefs_df[abs(coefs_df['Coef']) > 0.1].sort_values('Coef', ascending = False)

,Column,Coef
27,Pl1 Surface Performance,3.389839
25,Pl1 Perf. vs Similar Opponent,2.234437
19,Rank Index,0.362702
22,Pl1 Recent Form,0.313816
21,Pl0 Form,0.270823
20,Pl0 Recent Form,-0.272943
23,Pl1 Form,-0.523296
24,Pl0 Perf. vs Similar Opponent,-2.460449
26,Pl0 Surface Performance,-3.021576


We can see that the most important features are:
- Player performance on specific surface
- Player performance against similar ranked opponents
- The opponent's form
- The rank index
- The players recent form

Let's load our matches Dataset

In [10]:
df = pd.read_csv("csv/FeatureCalculated_Data.csv")

Now our goal is to build another Dataframe, containing the informations about the the players, who wins each match, the average and maximum odds available for each player, and we'll use the logistic regression coefficients (and the intercept) to calculate the predicted chances to win for each player:

In [11]:
betting_data_df = df[(df["Avg_Pl0"] > 1) & (df["Avg_Pl1"] > 1) & (df["Max_Pl0"] > 1) & (df["Max_Pl1"] > 1)]

betting_df_list = []
for ix, irow in betting_data_df.iterrows():

    current_linear_combination = lreg.intercept_[0]
    for attribute in list(coefs_map.keys()):
        attr_value = irow[attribute]
        attr_weighted = attr_value * coefs_map.get(attribute)
        current_linear_combination += attr_weighted
    
    prob_p1_win = round(exp(current_linear_combination) / (1 + exp(current_linear_combination)), 2)
    prob_p0_win = round(1 - prob_p1_win, 2)

    betting_df_list.append(   {"Date": irow['Date'], "Pl0":irow['Player 0'], "Pl1":irow['Player 1'], "Winner":irow['Won'], "Pl0 %":prob_p0_win, "Pl1 %":prob_p1_win, \
                              "Pl0 Avg odds": irow['Avg_Pl0'], "Pl1 Avg odds": irow['Avg_Pl1'], "Pl0 Max odds": irow['Max_Pl0'],
                               "Pl1 Max odds": irow['Max_Pl1']
                              })
    
betting_df = pd.DataFrame(betting_df_list)

In [12]:
betting_df.head()

,Date,Pl0,Pl1,Winner,Pl0 %,Pl1 %,Pl0 Avg odds,Pl1 Avg odds,Pl0 Max odds,Pl1 Max odds
0,2010-04-19,De Bakker T.,Falla A.,0.0,0.62,0.38,1.52,2.46,1.62,2.79
1,2010-04-19,Starace P.,Hajek J.,1.0,0.67,0.33,1.34,3.12,1.40,3.64
2,2010-04-19,Schwank E.,Fognini F.,0.0,0.33,0.67,1.99,1.77,2.14,1.85
3,2010-04-19,Rochus C.,Garcia-Lopez G.,1.0,0.33,0.67,3.02,1.36,3.25,1.45
4,2010-04-19,Bellucci T.,Nieminen J.,0.0,0.52,0.48,1.46,2.63,1.50,3.09


Let's define a helper funxtion that takes a change in input and returns the 'fair' odds for that chance value.

In [13]:
def right_odds(chance):
    return round(1/chance,2)

Now, we have everything we need to make decisions based on which player, for each match, we get the most value betting on!
In fact, next we're going to append to this betting dataframe another 4 columns.

The fair odds for both player 0 and player 1, based on the chances of each player to win according to the Logistic Regression model, and the value that we would get betting on each player at the max odds available!

In [14]:
for ix, row in betting_df.iterrows():
    right_odds_p0 = right_odds(row['Pl0 %'])
    right_odds_p1 = right_odds(row['Pl1 %'])
    bet_value_p0 = (row['Pl0 Max odds'] - right_odds_p0)/max(right_odds_p0,row['Pl0 Max odds'])
    bet_value_p1 = (row['Pl1 Max odds'] - right_odds_p1)/max(right_odds_p1,row['Pl1 Max odds'])
    betting_df.at[ix, 'Fair odds for Pl0'] = right_odds_p0
    betting_df.at[ix, 'Fair odds for Pl1'] = right_odds_p1
    betting_df.at[ix, 'Bet on Pl0 Value'] = bet_value_p0
    betting_df.at[ix, 'Bet on Pl1 Value'] = bet_value_p1

In [15]:
betting_df

,Date,Pl0,Pl1,Winner,Pl0 %,Pl1 %,Pl0 Avg odds,Pl1 Avg odds,Pl0 Max odds,Pl1 Max odds,Fair odds for Pl0,Fair odds for Pl1,Bet on Pl0 Value,Bet on Pl1 Value
0,2010-04-19,De Bakker T.,Falla A.,0.0,0.62,0.38,1.52,2.46,1.62,2.79,1.61,2.63,0.006173,0.057348
1,2010-04-19,Starace P.,Hajek J.,1.0,0.67,0.33,1.34,3.12,1.40,3.64,1.49,3.03,-0.060403,0.167582
2,2010-04-19,Schwank E.,Fognini F.,0.0,0.33,0.67,1.99,1.77,2.14,1.85,3.03,1.49,-0.293729,0.194595
3,2010-04-19,Rochus C.,Garcia-Lopez G.,1.0,0.33,0.67,3.02,1.36,3.25,1.45,3.03,1.49,0.067692,-0.026846
4,2010-04-19,Bellucci T.,Nieminen J.,0.0,0.52,0.48,1.46,2.63,1.50,3.09,1.92,2.08,-0.218750,0.326861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25349,2019-11-15,Nadal R.,Tsitsipas S.,0.0,0.26,0.74,1.41,2.93,1.48,3.30,3.85,1.35,-0.615584,0.590909
25350,2019-11-15,Medvedev D.,Zverev A.,1.0,0.02,0.98,1.90,1.92,2.06,2.24,50.00,1.02,-0.958800,0.544643
25351,2019-11-16,Tsitsipas S.,Federer R.,0.0,0.79,0.21,3.39,1.33,3.75,1.40,1.27,4.76,0.661333,-0.705882
25352,2019-11-16,Zverev A.,Thiem D.,1.0,0.25,0.75,2.06,1.78,2.20,1.87,4.00,1.33,-0.450000,0.288770


We just obtained 17104 matches where we can bet on the player who's value is higher!
Let's save this dataframe!

In [16]:
betting_df.to_csv("csv/Betting_Data.csv", index=False)